In [ ]:
import mnist
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv

In [ ]:
x_train = mnist.train_images()
y_train = mnist.train_labels()
x_test = mnist.test_images()
y_test = mnist.test_labels()

In [ ]:
def elongation(m):
    x = m['mu20'] + m['mu02']
    y = 4 * m['mu11']**2 + (m['mu20'] - m['mu02'])**2
    return (x + y**0.5) / (x - y**0.5)


def get_features(image):
    contours, _ = cv.findContours(image, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    cnt = contours[0]

    features = {}

    area = cv.contourArea(cnt)
    features['area'] = area
    perimeter = cv.arcLength(cnt, True)
    features['perimeter'] = perimeter
    try:
        features['perimeter_area_ratio'] = float(perimeter) / float(area)
    except:
        features['perimeter_area_ratio'] = 0

    x, y, w, h = cv.boundingRect(cnt)
    features['aspect_ratio'] = float(w) / float(h)
    
    rect_area = w*h
    features['extent'] = float(area) / float(rect_area)
    
    hull = cv.convexHull(cnt)
    hull_area = cv.contourArea(hull)
    features['hull_area'] = hull_area
    try:
        features['solidity'] = float(area) / float(hull_area)
    except:
        features['solidity'] = 0.0

    try:
        (x, y), (MA, ma), angle = cv.fitEllipse(cnt)
    except:
        angle, MA, ma = 0.0, 0.0, 0.0
    features['angle'] = angle
    features['major_axis_length'] = MA
    features['minor_axis_length'] = ma
    
    moments = cv.moments(image)
    features['elongation'] = elongation(moments)
    hu_moments = cv.HuMoments(moments)[0]
    
    c_moments = [moments[key] for key in ['m00', 'mu03', 'mu11', 'mu12', 'mu20', 'mu21', 'mu30']]
    cn_moments = [moments[key] for key in ['nu02', 'nu03', 'nu11', 'nu12', 'nu20', 'nu21', 'nu30']]

    return features, c_moments, cn_moments, hu_moments

In [ ]:
def convert(samples, prefix):
    all_features, all_c_moments, all_cn_moments, all_hu_moments = [], [], [], []
    for i, image in enumerate(samples):
        if i % 1000 == 0:
            print(i)
        
        features, c_moments, cn_moments, hu_moments = get_features(image)
        features = np.array([v[1] for v in sorted(features.items(), key=lambda x: x[0])])
        all_features.append(features)
        all_c_moments.append(c_moments)
        all_cn_moments.append(cn_moments)
        all_hu_moments.append(hu_moments)

    all_features = np.stack(all_features)
    all_c_moments = np.stack(all_c_moments)
    all_cn_moments = np.stack(all_cn_moments)
    all_hu_moments = np.stack(all_hu_moments)
    
    np.save(f'{prefix}_c_moments.npy', all_c_moments)
    np.save(f'{prefix}_cn_moments.npy', all_cn_moments)
    np.save(f'{prefix}_features.npy', all_features)
    np.save(f'{prefix}_hu_moments.npy', all_hu_moments)

convert(x_train, 'mnist_train')
convert(x_test, 'mnist_test')